In [1]:
import pyaudio
import speech_recognition as sr
import os
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain.chat_models import ChatOpenAI
os.environ['OPEN_API_KEY']='sk-proj-MgwdEZ7nwXLtf5poVrUrT3BlbkFJo77w3iUBszaxUHlVio3n'
import RPi.GPIO as GPIO
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import socket

**Trigger script**

Note : Rajouter fonction d'arrêt (mouvement)

In [ ]:
r = sr.Recognizer()

# Function to listen for the wake word
def listen_for_wake_word(wake_word="hey Carl"):
    with sr.Microphone() as source:
        print("Listening for wake word...")
        audio = r.listen(source)

    try:
        text = r.recognize_google(audio, language="en-US")
        if wake_word.lower() in text.lower():
            print("Wake word detected!")
            return True
        else:
            return False
    except sr.UnknownValueError:
        print("Could not understand audio")
    except sr.RequestError as e:
        print(f"Error; {e}")

    return False

# Function to execute after wake word is detected
def execute_script():
    print("Executing script...")
    # Add your script code here

# Main loop
while True:
    if listen_for_wake_word():
        execute_script()# Initialize the recognizer


**Speech-to-Text**

Note : Rajouter stop word

In [ ]:
with sr.Microphone() as source:
    print("Listening...")

    # Adjust for ambient noise levels
    r.adjust_for_ambient_noise(source)

    # Capture the audio input from the user
    audio = r.listen(source)
# Create a recognizer object
try:
    # Use the recognizer to convert speech to text
    text = r.recognize_google(audio)
    print("You said:", text)

    # Save the transcribed text to a file
    with open("transcript.txt", "w") as file:
        file.write(text + "\n")

except sr.UnknownValueError as e:
    print("Sorry, I could not understand your speech.")

except sr.RequestError as e:
    print("Sorry, an error occurred. Please check your internet connection.")

Listening...
You said: I would like you to go to the left then forward


**Context recognition**

Note : Rajouter trigger by response

In [ ]:
with open("transcript.txt", "r") as file:
    context_variable = file.read()

template = """Answer the question based only on the following context:

{context}

Question: {question}

You must only answer by "command" or "speech".

For your knowledge, here are some possible commands:
"left": "go to the left",
"right": "go to the right",
"forward": "go forward",
"ahead": "go forward",
"backward": "go backward",
"back": "go backward",
"u-turn": "make a U-turn",
"stop": "stop"
"return to your base": "return to base"
"""

prompt = PromptTemplate(
    input_variables=["context","question"],
    template=template
)



llm = ChatOpenAI(model="gpt-4-1106-preview",temperature=0)
# The prompt here should take as an input variable the
# `document_variable_name`

llm_chain = LLMChain(llm=llm, prompt=prompt)

query = "Is this transcripted text about a movement command or a speech ?"
res1 = llm_chain.run(context=context_variable, question=query)
print(res1)

if res1 == "command":
    # Define your commands
    commands = ["left", "right", "forward", "ahead", "backward", "back", "u-turn", "stop", "return to your base"]

    # Split the context into words
    words = context_variable.split()

    # Filter the words to get only the commands
    command_list = [word for word in words if word in commands]

    print(command_list)

if res1 == "speech":
    print(context_variable)


command
['left', 'forward']


**Segmentation between Text-to-command and Text summary**

In [ ]:
class CommandExecutor:
    def __init__(self):
        # Pin setup
        self.motorLeftA = 17
        self.motorLeftB = 18
        self.motorRightA = 22
        self.motorRightB = 23

        # Set up GPIO pins
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.motorLeftA, GPIO.OUT)
        GPIO.setup(self.motorLeftB, GPIO.OUT)
        GPIO.setup(self.motorRightA, GPIO.OUT)
        GPIO.setup(self.motorRightB, GPIO.OUT)

    # Function to control motors
    def control_motors(self, leftA, leftB, rightA, rightB, duration=None):
        GPIO.output(self.motorLeftA, leftA)
        GPIO.output(self.motorLeftB, leftB)
        GPIO.output(self.motorRightA, rightA)
        GPIO.output(self.motorRightB, rightB)
        if duration:
            time.sleep(duration)
            self.stop_motors()

    def stop_motors(self):
        GPIO.output(self.motorLeftA, False)
        GPIO.output(self.motorLeftB, False)
        GPIO.output(self.motorRightA, False)
        GPIO.output(self.motorRightB, False)

    # Command functions
    def go_left(self, duration=1):
        self.control_motors(True, False, False, True, duration)

    def go_right(self, duration=1):
        self.control_motors(False, True, True, False, duration)

    def go_forward(self, duration=1):
        self.control_motors(True, False, True, False, duration)

    def go_backward(self, duration=1):
        self.control_motors(False, True, False, True, duration)

    def make_u_turn(self, duration=2):
        self.go_right(duration)

    def return_to_base(self):
        # Example sequence to return to base
        print("Returning to base...")
        self.go_backward(2)
        self.go_left(1)
        self.go_forward(3)
        self.stop()
        print("Arrived at base")

    def stop(self):
        self.stop_motors()

    def execute_command(self, command):
        # Dictionary mapping commands to functions
        commands = {
            "left": self.go_left,
            "right": self.go_right,
            "forward": self.go_forward,
            "ahead": self.go_forward,
            "backward": self.go_backward,
            "back": self.go_backward,
            "u-turn": self.make_u_turn,
            "stop": self.stop,
            "return to base": self.return_to_base
        }

        if command in commands:
            commands[command]()
        else:
            print("Unknown command:", command)


class SpeechProcessor:
    def process_speech(self):
        with open("transcript.txt", "r") as file:
            context_variable = file.read()

            template = """Synthetise the following conversation:

            {context}
            You must capable of:
            - Understand the context of the conversation. 
                Example 1: A patient requires medical attention.
                Example 2: A doctor is talking to a patient.
                Example 3: A doctor is giving you information.
            - Differenciate the different persons in the conversation.
            """

            prompt = PromptTemplate(
             input_variables=["context"],
             template=template
            )

            llm = ChatOpenAI(model="gpt-4-1106-preview",temperature=0)
                # The prompt here should take as an input variable the
                # `document_variable_name`

            llm_chain = LLMChain(llm=llm, prompt=prompt)
            
            res_speech = llm_chain.run(context=context_variable)
            # Save the synthesized speech to a file
            with open("synthesized_speech.txt", "w") as file:
                file.write(res_speech)


command_executor = CommandExecutor()
speech_processor = SpeechProcessor()

if res1 == "command":
        for command in command_list:
            command_executor.execute_command(command)
elif res1 == "speech":
        speech_processor.process_speech()
        def send_email():
            # Email configuration
            sender_email = "arthur.perrot@epitech.digital"
            receiver_email = "arthur.perrot@epitech.digital"
            subject = "Synthesized Speech"
            message = "Please find the synthesized speech attached."

            # Create a multipart message
            email_message = MIMEMultipart()
            email_message["From"] = sender_email
            email_message["To"] = receiver_email
            email_message["Subject"] = subject

            # Attach the synthesized speech file
            with open("synthesized_speech.txt", "r") as file:
                attachment = MIMEText(file.read())
                attachment.add_header("Content-Disposition", "attachment", filename="synthesized_speech.txt")
                email_message.attach(attachment)

            # Send the email
            with smtplib.SMTP("smtp.gmail.com", 587) as server:
                server.starttls()
                server.login(sender_email, "your_password")
                server.send_message(email_message)

        # Call the send_email function
        send_email()


In [ ]:
# class CommandExecutor:
#     def __init__(self):
#         # Pin setup
#         self.motorLeftA = 17
#         self.motorLeftB = 18
#         self.motorRightA = 22
#         self.motorRightB = 23

#         # Set up GPIO pins
#         GPIO.setmode(GPIO.BCM)
#         GPIO.setup(self.motorLeftA, GPIO.OUT)
#         GPIO.setup(self.motorLeftB, GPIO.OUT)
#         GPIO.setup(self.motorRightA, GPIO.OUT)
#         GPIO.setup(self.motorRightB, GPIO.OUT)

#     # Function to control motors
#     def control_motors(self, leftA, leftB, rightA, rightB, duration=None):
#         GPIO.output(self.motorLeftA, leftA)
#         GPIO.output(self.motorLeftB, leftB)
#         GPIO.output(self.motorRightA, rightA)
#         GPIO.output(self.motorRightB, rightB)
#         if duration:
#             time.sleep(duration)
#             self.stop_motors()

#     def stop_motors(self):
#         GPIO.output(self.motorLeftA, False)
#         GPIO.output(self.motorLeftB, False)
#         GPIO.output(self.motorRightA, False)
#         GPIO.output(self.motorRightB, False)

#     # Command functions
#     def go_left(self, duration=1):
#         self.control_motors(True, False, False, True, duration)

#     def go_right(self, duration=1):
#         self.control_motors(False, True, True, False, duration)

#     def go_forward(self, duration=1):
#         self.control_motors(True, False, True, False, duration)

#     def go_backward(self, duration=1):
#         self.control_motors(False, True, False, True, duration)

#     def make_u_turn(self, duration=2):
#         self.go_right(duration)

#     def return_to_base(self):
#         # Example sequence to return to base
#         print("Returning to base...")
#         self.go_backward(2)
#         self.go_left(1)
#         self.go_forward(3)
#         self.stop()
#         print("Arrived at base")

#     def stop(self):
#         self.stop_motors()

#     def execute_command(self, command):
#         # Dictionary mapping commands to functions
#         commands = {
#             "left": self.go_left,
#             "right": self.go_right,
#             "forward": self.go_forward,
#             "ahead": self.go_forward,
#             "backward": self.go_backward,
#             "back": self.go_backward,
#             "u-turn": self.make_u_turn,
#             "stop": self.stop,
#             "return to base": self.return_to_base
#         }

#         if command in commands:
#             commands[command]()
#         else:
#             print("Unknown command:", command)


# class SpeechProcessor:
#     def process_speech(self):
#         with open("transcript.txt", "r") as file:
#             context_variable = file.read()

#             template = """Synthetise the following conversation:

#             {context}
#             You must capable of:
#             - Understand the context of the conversation. 
#                 Example 1: A patient requires medical attention.
#                 Example 2: A doctor is talking to a patient.
#                 Example 3: A doctor is giving you information.
#             - Differenciate the different persons in the conversation.
#             """

#             prompt = PromptTemplate(
#              input_variables=["context"],
#              template=template
#             )

#             llm = ChatOpenAI(model="gpt-4-1106-preview",temperature=0)
#                 # The prompt here should take as an input variable the
#                 # `document_variable_name`

#             llm_chain = LLMChain(llm=llm, prompt=prompt)
            
#             res_speech = llm_chain.run(context=context_variable)


# command_executor = CommandExecutor()
# speech_processor = SpeechProcessor()

# # for res1 in [res1]:
# if res1 == "command":
#         for command in command_list:
#             command_executor.execute_command(command)
# elif res1 == "speech":
#         speech_processor.process_speech()

NameError: name 'GPIO' is not defined

Rajouter command de déplacement 360 pour acknowledge fin de mission